# Inference

In [ ]:
from transformers import XLMRobertaForTokenClassification, XLMRobertaTokenizerFast

model_load_path = "./ner_model"
loaded_model = XLMRobertaForTokenClassification.from_pretrained(model_load_path)
loaded_tokenizer = XLMRobertaTokenizerFast.from_pretrained(model_load_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)

In [ ]:
from transformers import XLMRobertaForTokenClassification, XLMRobertaTokenizerFast

model_load_path = "/content/clients_file"
loaded_model = XLMRobertaForTokenClassification.from_pretrained(model_load_path)
loaded_tokenizer = XLMRobertaTokenizerFast.from_pretrained(model_load_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)

In [9]:
import torch
import json

ner_labels = loaded_model.config.id2label.values()

def perform_ner(text, model, tokenizer, id2label):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted label indices
    predictions = torch.argmax(outputs.logits, dim=2)

    # Get the confidence scores
    confidence_scores = torch.nn.functional.softmax(outputs.logits, dim=2)

    # Decode the predictions
    decoded_preds = []
    for token, pred, conf in zip(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]),
                                 predictions[0],
                                 confidence_scores[0]):
        if token.startswith("▁"):  # XLM-RoBERTa uses '▁' to denote start of word
            label = id2label[pred.item()]
            confidence = conf[pred].item() * 100  # Convert to percentage
            decoded_preds.append({
                "token": token.lstrip("▁"),
                "label": label,
                "confidence": f"{confidence:.2f}%"
            })

    # Create JSON-like response
    response = {}
    for pred in decoded_preds:
        if pred['label'] != 'O':  # Ignore 'Outside' labels
            entity_type = pred['label'].split('-')[-1]  # Get the entity type (e.g., 'Vehicle' from 'B-Vehicle')
            if entity_type not in response:
                response[entity_type] = []
            response[entity_type].append({
                "token": pred['token'],
                "label": pred['label'],
                "confidence": pred['confidence']
            })

    return json.dumps(response, indent=2)

# Create id2label mapping
id2label = {i: label for i, label in enumerate(ner_labels)}

In [10]:
# Example usage
text = "Hulls señaló que en el sistema jurídico de la Commonwealth , en el que se basa la justicia australiana , es fundamental que la persona sea juzgada únicamente teniendo en cuenta las pruebas presentadas ante el juez ."
result = perform_ner(text, loaded_model, loaded_tokenizer, id2label)
print(result)

{
  "PER": [
    {
      "token": "Hu",
      "label": "B-PER",
      "confidence": "99.72%"
    }
  ],
  "ORG": [
    {
      "token": "Common",
      "label": "B-ORG",
      "confidence": "97.46%"
    }
  ]
}
